# Homework 3

Austin Gill

Problems:

* 5: 2, 10abd
* 6: 5, 8
* Text addition

---

In [ ]:
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

## Problem 5.2

Using Veranda and the basic motion algorithm, place a set of obstacles that cause the robot to cycle and not find the goal. In other words, build a robot trap.

## Problem 5.10

In Veranda and Python, implement

1. Bug 1
2. Bug 2
3. Tangent Bug

Screen capture the result.

## Problem 6.5
Real motion and measurement involves error and this problem will introduce the concepts. Assume that you have a differential drive robot with wheels that are 20cm in radius and $L$ is 12cm. Using the differential drive code (forward kinematics) from the text, develop code to simulate the robot motion when the wheel velocities are $\dot\phi_1 = 0.25 t^2$, $\dot\phi_2 = 0.5t$. The starting location is $(0, 0)$ with $\theta = 0$.

1. Plot the path of the robot on $0 \leq t \leq 5$. It should end up somewhere near $(50, 60)$.

2. Assume that you have Gaussian noise added to the omegas each time you evaluate the velocity (each time step). Test with $\mu=0$ and $\sigma=0.3$. Write the final location $(x,y)$ to a file and repeat for 100 simulations. Hint:

  ```python
  mu, sigma = 0.0, 0.3
  xerr = np.random.normal(mu,sigma, NumP)
  yerr = np.random.normal(mu,sigma, NumP)
  ```

3. Generate a plot that includes the noise free robot path and the final locations for the simulations with noise. Hint:

  ```python
  import numpy as np
  import pylab as plt
  ...
  plt.plot(xpath,ypath, 'b-', x,y, 'r.')
  plt.xlim(-10, 90)
  plt.ylim(-20, 80)
  plt.show()
  ```

4. Find the location means and 2x2 covariance matrix for this data set, and compute the eigenvalues and eigenvectors of the matrix. Find the ellipse that these generate. [The major and minor axes directions are given by the eigenvectors. Show the point cloud of final locations and the ellipse in a graphic (plot the data and the ellipse). Hint:

  ```python
  from scipy import linalg
  from matplotlib.patches import Ellipse

  # assume final locations are in x & y
  mat = np.array([x,y])
  # find covariance matrix
  cmat = np.cov(mat)
  # compute eigenvals and eigenvects of covariance
  eval, evec = linalg.eigh(cmat)
  # find ellipse rotation angle
  angle = 180 * atan2(evec[0, 1], evec[0, 0]) / np.pi
  # create ellipse
  ell = Ellipse((np.mean(x), np.mean(y)), eval[0], eval[1], angle)
  # make the ellipse subplot
  a = plt.subplot(111, aspect='equal')
  ell.set_alpha(0.1) # make the ellipse lighter
  a.add_artist(ell)  # add this to the plot
  ```

## Problem 6.10
In Veranda, drive a mecanum robot in an $\infty$ symbol. Record the results.